In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!setsid nohup ollama serve > run_serve.txt 2>&1 &

In [3]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏  21 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  40 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  57 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏  89 MB/4.7 GB                  pulling ma

In [4]:
!setsid ollama run llama3 > run_llama.txt 2>&1 &

In [5]:
!ollama list

NAME         	ID          	SIZE  	MODIFIED               
llama3:latest	365c0bd3c000	4.7 GB	Less than a second ago	


## LLM + Prompt

In [ ]:
# !pip install langchain
# !pip install langchain_community

In [28]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3")

llm.invoke("在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？")

"I think there may be some misinformation here. 🤔\n\nTo date, no such incident has occurred in 2023 or 2024 involving former President Donald Trump being shot at a campaign rally. While Trump has been involved in several controversies and incidents during his presidency and post-presidency, this specific scenario has not happened.\n\nSo, I can't provide an answer about the location of such an event because it hasn't occurred. 🙅\u200d♂️"

In [29]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一名新闻记者，很熟悉最近的重大新闻事件。"),
    ("user", "{input}")
])

chain = prompt | llm

chain.invoke({"input": "在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？"})

"I'm a journalist, and I can tell you that the incident you're referring to is a shocking and tragic event. According to reports, former President Donald Trump was indeed attacked with gunfire at a campaign rally in Tuscawilla, Florida, on February 12, 2024.\n\nAs per eyewitness accounts and official investigations, Trump was speaking at a packed auditorium when suddenly, a gunman opened fire from the balcony. The attack left several people injured, including Trump himself, who sustained minor injuries to his arm.\n\nThe FBI launched an investigation into the incident, which led to the arrest of the suspected shooter, identified as 32-year-old James T. Reed of Orlando. Reed was reportedly motivated by political ideology and had a history of making threatening comments online.\n\nThis heinous attack sent shockwaves across the nation, prompting widespread condemnation and renewed calls for stricter gun control measures. As a journalist, I've been covering this story closely, and it's cle

## RAG Retrieval Chain

In [ ]:
# !pip install beautifulsoup4
# !pip install faiss-gpu

In [30]:
# 划分 chunks

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://www.thepaper.cn/newsDetail_forward_28067225")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100, add_start_index=True)

documents = text_splitter.split_documents(docs)

print(len(documents))
print(documents[0])

6
page_content='特朗普遭枪击细节披露：枪手距离约200米，用步枪打了8发子弹！子弹划过特朗普耳朵画面曝光_澎湃号·媒体_澎湃新闻-The Paper下载客户端登录无障碍+1特朗普遭枪击细节披露：枪手距离约200米，用步枪打了8发子弹！子弹划过特朗普耳朵画面曝光2024-07-14 10:04来源：澎湃新闻·澎湃号·媒体字号据新华社最新报道：美国特勤局13日通报说，一名枪手向特朗普集会演讲台开数枪，该枪手已被击毙，集会观众中1人死亡2人受伤，事件正在调查中。这张视频截图显示，7月13日，美国前总统特朗普在宾夕法尼亚州巴特勒市的竞选集会现场被护送离开特朗普称右耳上部中枪美警方称其被提词器碎片击中当地时间13日，' metadata={'source': 'https://www.thepaper.cn/newsDetail_forward_28067225', 'title': '特朗普遭枪击细节披露：枪手距离约200米，用步枪打了8发子弹！子弹划过特朗普耳朵画面曝光_澎湃号·媒体_澎湃新闻-The Paper', 'description': '据新华社最新报道：美国特勤局13日通报说，一名枪手向特朗普集会演讲台开数枪，该枪手已被击毙，集会观众中1人死亡2人受伤，事件正在调查中。这张视频截图显示，7月1', 'language': 'No language found.', 'start_index': 0}


In [31]:
# vector 搜索相似性

from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OllamaEmbeddings(model="llama3")

vector = FAISS.from_documents(documents, embeddings)

question = "在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？"

# 默认参数：k=4
search_results = vector.similarity_search(question)

print(search_results)

[Document(metadata={'source': 'https://www.thepaper.cn/newsDetail_forward_28067225', 'title': '特朗普遭枪击细节披露：枪手距离约200米，用步枪打了8发子弹！子弹划过特朗普耳朵画面曝光_澎湃号·媒体_澎湃新闻-The Paper', 'description': '据新华社最新报道：美国特勤局13日通报说，一名枪手向特朗普集会演讲台开数枪，该枪手已被击毙，集会观众中1人死亡2人受伤，事件正在调查中。这张视频截图显示，7月1', 'language': 'No language found.', 'start_index': 1099}, page_content='，这是不合适的，”拜登在特拉华州发表讲话时说，“每个人都必须谴责它。每个人。”拜登补充说，“联邦政府的每个机构”都在调查此事，并向他提供最新情况的报告。在发表讲话前，拜登在社交平台X上发文称，他很庆幸特朗普在宾夕法尼亚州集会上遭遇枪击后平安无事。“我已听取了有关唐纳德·特朗普在宾夕法尼亚州集会上枪击事件的汇报。我很庆幸听说他安全并且情况良好。”他补充道：“我正在为他和他的家人以及所有参加集会的人祈祷，我们正在等待进一步的消息。吉尔（拜登夫人）和我特此感谢特勤局确保了特朗普的安全。美国不允许这种暴力行为发生。我们必须团结起来，作为一个国家谴责这种行为。”原标题：《特朗普遭枪击细节披露：枪手距离约200米，用步枪打了8发子弹！子弹划过特朗普耳朵画面曝光》阅读原文特别声明本文为澎湃号作者或机构在澎湃新闻上传并发布，仅代表该作者或机构观点，不代表澎湃新闻的观点或立场，澎湃新闻仅提供信息发布平台。申请澎湃号请用电脑访问http://renzheng.thepaper.cn。+1收藏我要举报查看更多查看更多开始答题扫码下载澎湃新闻客户端Android版iPhone版iPad版关于澎湃加入澎湃联系我'), Document(metadata={'source': 'https://www.thepaper.cn/newsDetail_forward_28067225', 'title': '特朗普遭枪击细节披露：枪手距离约200米，用步枪打了8发子弹！子弹划过特朗普耳朵画面曝光_澎湃号·媒体_澎湃新闻-The Paper', 

In [32]:
print(len(search_results))

4


In [33]:
search_content = search_results[0].page_content

print(search_content)

，这是不合适的，”拜登在特拉华州发表讲话时说，“每个人都必须谴责它。每个人。”拜登补充说，“联邦政府的每个机构”都在调查此事，并向他提供最新情况的报告。在发表讲话前，拜登在社交平台X上发文称，他很庆幸特朗普在宾夕法尼亚州集会上遭遇枪击后平安无事。“我已听取了有关唐纳德·特朗普在宾夕法尼亚州集会上枪击事件的汇报。我很庆幸听说他安全并且情况良好。”他补充道：“我正在为他和他的家人以及所有参加集会的人祈祷，我们正在等待进一步的消息。吉尔（拜登夫人）和我特此感谢特勤局确保了特朗普的安全。美国不允许这种暴力行为发生。我们必须团结起来，作为一个国家谴责这种行为。”原标题：《特朗普遭枪击细节披露：枪手距离约200米，用步枪打了8发子弹！子弹划过特朗普耳朵画面曝光》阅读原文特别声明本文为澎湃号作者或机构在澎湃新闻上传并发布，仅代表该作者或机构观点，不代表澎湃新闻的观点或立场，澎湃新闻仅提供信息发布平台。申请澎湃号请用电脑访问http://renzheng.thepaper.cn。+1收藏我要举报查看更多查看更多开始答题扫码下载澎湃新闻客户端Android版iPhone版iPad版关于澎湃加入澎湃联系我


In [34]:
search_content = search_results[1].page_content

print(search_content)

SHANGHAISIXTH TONE新闻报料报料热线: 021-962866报料邮箱: news@thepaper.cn沪ICP备14003370号沪公网安备31010602000299号互联网新闻信息服务许可证：31120170006增值电信业务经营许可证：沪B2-2017116© 2014-2024 上海东方报业有限公司反馈


In [35]:
search_content = search_results[2].page_content

print(search_content)

特朗普遭枪击细节披露：枪手距离约200米，用步枪打了8发子弹！子弹划过特朗普耳朵画面曝光_澎湃号·媒体_澎湃新闻-The Paper下载客户端登录无障碍+1特朗普遭枪击细节披露：枪手距离约200米，用步枪打了8发子弹！子弹划过特朗普耳朵画面曝光2024-07-14 10:04来源：澎湃新闻·澎湃号·媒体字号据新华社最新报道：美国特勤局13日通报说，一名枪手向特朗普集会演讲台开数枪，该枪手已被击毙，集会观众中1人死亡2人受伤，事件正在调查中。这张视频截图显示，7月13日，美国前总统特朗普在宾夕法尼亚州巴特勒市的竞选集会现场被护送离开特朗普称右耳上部中枪美警方称其被提词器碎片击中当地时间13日，


In [36]:
search_content = search_results[3].page_content

print(search_content)

情况安全。拜登竞选团队宣布暂停接下来的所有外出活动美国总统拜登在收到消息通报后表示，很高兴听到特朗普安全并情况良好。感谢特勤局将特朗普安全送达医院。美国不会容忍这种暴力行为。当日，美国总统拜登竞选团队宣布暂停拜登所有接下来的外出活动，并将尽快撤下所有的电视竞选广告。拜登发表全国讲话据美国有线电视新闻网（CNN）最新消息，美国前总统特朗普集会上发生枪击事件后，美国总统拜登发表了全国讲话。他在讲话中谴责这起枪击事件，并表示“美国不应该容忍这种暴力行为。”“这太令人厌恶了，这是我们必须将这个国家团结起来的原因之一，”他在特拉华州发表讲话时说道，“我们不能允许这种事情发生。我们不能这样，我们不能纵容这种事情。”拜登补充说，他试图与特朗普取得联系，并提到特朗普正在接受医生治疗，似乎身体情况良好。“最重要的是，特朗普集会……应该能够和平进行，不发生任何问题。美国发生这样的暴力或者政治暴力，是闻所未闻的，这是不合适的，”拜登在特拉华州发表讲话时说，“每个人都必须谴责它。每个人。”拜登补充说，“联邦政府的每个机构”都在调查此事，并向他提供最新情况的报告。在发表讲话前，拜登在社交平台X上发文称，他很庆幸


In [37]:
# 默认参数 similarity，k=4
retriever = vector.as_retriever()

llm = Ollama(model="llama3")

prompt = ChatPromptTemplate.from_template(
    """根据上下文的信息，回答指定问题:
    <context>
    {context}
    </context>
    Question: {input}
    """)

from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke(
    {"input": "在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？"})

print(response["answer"])

根据上下文信息，我们可以知道，特朗普在2024年遭到枪击的地点是宾夕法尼亚州巴特勒市。


In [38]:
# 换一个问题

response = retrieval_chain.invoke(
    {"input": "拜登竞选团队在枪击事件后怎么做？"})

print(response["answer"])


根据上下文信息，拜登竞选团队在枪击事件后宣布暂停接下来的所有外出活动，并将尽快撤下所有的电视竞选广告。


In [39]:
# 换一个问题

response = retrieval_chain.invoke(
    {"input": "特朗普在枪击事件后还幸存吗？"})

print(response["answer"])

根据上下文信息，可以确定特朗普在枪击事件后仍然幸存。具体来说，文中写道：“美警方称其被提词器碎片击中当地时间13日，情况安全。”同时，也有拜登的讲话记录，特朗普在接受医生治疗，身体情况良好。这表明，特朗普在枪击事件后并没有遇到生命危险或死亡，因此可以确定他仍然幸存。


## Conversation Retrieval Chain with History

In [40]:
from langchain_core.prompts import MessagesPlaceholder

from langchain.chains import create_history_aware_retriever

from langchain_core.messages import HumanMessage, AIMessage

In [43]:
# Answer the user's questions

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("system", "根据上下文的信息，回答指定问题：\n\n{context}"),
])

llm = Ollama(model="llama3")

retriever = vector.as_retriever()

# 创建历史检索链
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

chat_history = [
    HumanMessage(content="在2024年，特朗普在竞选集会现场遭到枪击，请问发生在哪个地点？"),
    AIMessage(content="根据上下文信息，我们可以知道，特朗普在2024年遭到枪击的地点是宾夕法尼亚州巴特勒市。")
]

response = retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "特朗普被刺杀后，现在的情况怎么样？"
})

print(response["answer"])

根据上下文信息，特朗普在宾夕法尼亚州的竞选集会现场遭到枪击事件，但他身体情况良好，并正在接受医生治疗。事件中有1人死亡、2人受伤，枪手已被击毙。美国总统拜登在发表讲话时谴责这起枪击事件，表示“每个人都必须谴责它。每个人。”他也感谢特勤局将特朗普安全送达医院，并补充说，“联邦政府的每个机构”都在调查此事，并向他提供最新情况的报告。
